In [1]:
import pandas as pd
from basilik import Node, BN

In [3]:
# define nodes/edges

C = Node("cloudy")
R = Node("rain", [C])
S = Node("sprinkler", [C])
W = Node("wet", [R, S])

ls_n = [C, R, S, W]

In [5]:
# fetch joint observations
obs = pd.read_csv("data/observations.csv").drop("Unnamed: 0", axis=1)
obs.head()

,cloudy,rain,sprinkler,wet
0,True,True,False,True
1,False,False,False,False
2,False,True,False,True
3,False,False,False,False
4,False,False,False,False


In [6]:
# construct model with nodes and observations
model = BN(ls_n, obs)

In [7]:
# easily view conditional probabilities for each node
model.generate_cpt("sprinkler")

sprinkler,cloudy,False,True
0,False,0.496855,0.503145
1,True,0.892925,0.107075


In [8]:
# easily view conditional probabilities for each node
model.generate_cpt("wet")

wet,rain,sprinkler,False,True
0,False,False,1.000000,0.000000
1,False,True,0.113300,0.886700
2,True,False,0.099291,0.900709
3,True,True,0.010753,0.989247
